- Codes are from [GPT-2 with HuggingFace + PyTorch](https://www.kaggle.com/code/baekseungyun/gpt-2-with-huggingface-pytorch)

In [ ]:
!pip install transformers

In [ ]:
from transformers import set_seed, GPT2Config, GPT2Tokenizer, GPT2ForSequenceClassification

set_seed(42) 

model_config = GPT2Config.from_pretrained('gpt2', num_labels=7)
model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=model_config)

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = model.config.eos_token_id

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
import pandas as pd

pd.read_csv("./data_info.csv", encoding="cp949")

,feature,information,type
0,ID,고유번호,object
1,Utterance,발화문,object
2,Speaker,발화자,object
3,Dialogue_ID,Dialogue 구분 번호,int
4,Target,감정,object


In [ ]:
import os
import pandas as pd
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, train=True):
        super().__init__()
        self.train = train
        self.data = pd.read_csv(os.path.join('./', 'train.csv' if train else 'test.csv'))
        if self.train:
          self.data['Target'] = self.data['Target'].replace({'neutral':0, 'surprise':1, 'fear':2, 'sadness':3, 'joy':4, 'disgust':5, 'anger':6})

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        record = self.data.iloc[index]       
        text = record['Utterance']
        if self.train:
            return {'text': text, 'label': record['Target']}
        else:
            return {'text': text, 'label': '0'}

train_dataset = CustomDataset(train=True)
test_dataset = CustomDataset(train=False)

In [ ]:
class Gpt2ClassificationCollator(object):
    def __init__(self, tokenizer, max_seq_len=None):
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        
        return
    
    def __call__(self, sequences):
        texts = [sequence['text'] for sequence in sequences]
        labels = [int(sequence['label']) for sequence in sequences]
        inputs = self.tokenizer(text=texts,
                                return_tensors='pt',
                                padding=True,
                                truncation=True,
                                max_length=self.max_seq_len)
        inputs.update({'labels': torch.tensor(labels)})
        
        return inputs

gpt2classificationcollator = Gpt2ClassificationCollator(tokenizer=tokenizer, max_seq_len=256)

In [ ]:
from torch.utils.data import DataLoader, random_split

train_size = int(len(train_dataset) * 0.8)
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=16,
                              shuffle=True,
                              collate_fn=gpt2classificationcollator)

val_dataloader = DataLoader(dataset=val_dataset,
                            batch_size=16,
                            shuffle=False,
                            collate_fn=gpt2classificationcollator)

test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=16,
                             shuffle=False,
                             collate_fn=gpt2classificationcollator)

In [ ]:
from transformers import AdamW, get_cosine_schedule_with_warmup

total_epochs = 50

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters,
                  lr=1e-5,
                  eps=1e-8)

num_train_steps = len(train_dataloader) * total_epochs
num_warmup_steps = int(num_train_steps * 0.1) 

lr_scheduler = get_cosine_schedule_with_warmup(optimizer,
                                               num_warmup_steps=num_warmup_steps,
                                               num_training_steps = num_train_steps)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import torch

def train(dataloader, optimizer, scheduler, device_):
    global model
    model.train()
    
    prediction_labels = []
    true_labels = []
    
    total_loss = []
    
    for batch in dataloader:
        true_labels += batch['labels'].numpy().flatten().tolist()
        batch = {k:v.type(torch.long).to(device_) for k, v in batch.items()}
        
        
        outputs = model(**batch)
        loss, logits = outputs[:2]
        logits = logits.detach().cpu().numpy()
        total_loss.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # prevent exploding gradient

        optimizer.step()
        scheduler.step()
        
        prediction_labels += logits.argmax(axis=-1).flatten().tolist()
    
    return true_labels, prediction_labels, total_loss

def validation(dataloader, device_):
    global model
    model.eval()
    
    prediction_labels = []
    true_labels = []
    
    total_loss = []
    
    for batch in dataloader:
        true_labels += batch['labels'].numpy().flatten().tolist()
        batch = {k:v.type(torch.long).to(device_) for k, v in batch.items()}
        
        with torch.no_grad():
            outputs = model(**batch)
            loss, logits = outputs[:2]
            logits = logits.detach().cpu().numpy()
            total_loss.append(loss.item())

            prediction_labels += logits.argmax(axis=-1).flatten().tolist()
        
    return true_labels, prediction_labels, total_loss

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

all_loss = {'train_loss': [], 'val_loss': []}
all_acc = {'train_acc': [], 'val_acc': []}

for epoch in range(total_epochs):
    y, y_pred, train_loss = train(train_dataloader, optimizer, lr_scheduler, device)
    train_acc = accuracy_score(y, y_pred)
    
    y, y_pred, val_loss = validation(val_dataloader, device)
    val_acc = accuracy_score(y, y_pred)
    
    all_loss['train_loss'] += train_loss
    all_loss['val_loss'] += val_loss
    
    all_acc['train_acc'].append(train_acc)
    all_acc['val_acc'].append(val_acc)
    
    print(f'Epoch: {epoch}, train_loss: {torch.tensor(train_loss).mean():.3f}, train_acc: {train_acc:.3f}, val_loss: {torch.tensor(val_loss).mean():.3f}, val_acc: {val_acc:.3f}') 

Epoch: 0, train_loss: 3.175, train_acc: 0.178, val_loss: 2.492, val_acc: 0.390
Epoch: 1, train_loss: 2.805, train_acc: 0.333, val_loss: 2.517, val_acc: 0.422
Epoch: 2, train_loss: 2.667, train_acc: 0.337, val_loss: 2.232, val_acc: 0.429
Epoch: 3, train_loss: 2.283, train_acc: 0.374, val_loss: 1.859, val_acc: 0.467
Epoch: 4, train_loss: 1.983, train_acc: 0.410, val_loss: 1.667, val_acc: 0.474
Epoch: 5, train_loss: 1.823, train_acc: 0.432, val_loss: 1.641, val_acc: 0.486
Epoch: 6, train_loss: 1.771, train_acc: 0.439, val_loss: 1.610, val_acc: 0.485
Epoch: 7, train_loss: 1.741, train_acc: 0.446, val_loss: 1.592, val_acc: 0.490
Epoch: 8, train_loss: 1.715, train_acc: 0.453, val_loss: 1.591, val_acc: 0.492
Epoch: 9, train_loss: 1.692, train_acc: 0.460, val_loss: 1.572, val_acc: 0.494
Epoch: 10, train_loss: 1.655, train_acc: 0.470, val_loss: 1.577, val_acc: 0.496
Epoch: 11, train_loss: 1.663, train_acc: 0.453, val_loss: 1.551, val_acc: 0.478
Epoch: 12, train_loss: 1.539, train_acc: 0.479, va

In [ ]:
_, y_pred, _ = validation(test_dataloader, device)

In [ ]:
y_pred[0]

0

In [ ]:
for i in range(len(y_pred)):
    if int(y_pred[i]) == 0 :
        y_pred[i] = 'neutral'
    elif int(y_pred[i]) == 1 :
        y_pred[i] = 'surprise'
    elif int(y_pred[i]) == 2 :
        y_pred[i] = 'fear'
    elif int(y_pred[i]) == 3 :
        y_pred[i] = 'sadness'
    elif int(y_pred[i]) == 4 :
        y_pred[i] = 'joy'
    elif int(y_pred[i]) == 5 :
        y_pred[i] = 'disgust'
    elif int(y_pred[i]) == 6 :
        y_pred[i] = 'anger'

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['Target'] = y_pred
submit.to_csv('./GPT-2_submission.csv', index=False)